#### Link of the Pycaret docs : https://pycaret.readthedocs.io/en/latest/api/time_series.html

In [66]:
#importing necessary packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import numpy as np 
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import imageio
import os
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit
from pycaret.time_series import TSForecastingExperiment
import pmdarima as pm


In [67]:
#função para tratar campo data
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('dbmilho_br.xls', sheet_name='Planilha1')
print(df.head(10))

   date  producao  produtividade     area
0  1977   19256.0           1632  11797.3
1  1978   14017.0           1276  10985.1
2  1979   16514.0           1461  11304.8
3  1980   19435.0           1665  11669.9
4  1981   21284.0           1752  12147.1
5  1982   21605.0           1692  12771.1
6  1983   19015.0           1631  11658.2
7  1984   21178.0           1735  12205.6
8  1985   21175.0           1773  11940.2
9  1986   20265.0           1549  13083.5


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           48 non-null     int64  
 1   producao       48 non-null     float64
 2   produtividade  48 non-null     int64  
 3   area           47 non-null     float64
dtypes: float64(2), int64(2)
memory usage: 1.6 KB


### Avaliando a produção de soja

In [69]:
df.drop(['produtividade','area'], axis = 1,inplace=True)
df.head(10)


,date,producao
0,1977,19256.0
1,1978,14017.0
2,1979,16514.0
3,1980,19435.0
4,1981,21284.0
5,1982,21605.0
6,1983,19015.0
7,1984,21178.0
8,1985,21175.0
9,1986,20265.0


In [70]:
df.set_index('date',inplace=True)

In [71]:
df=df.sort_index(axis=0)
print(df.head(10))

      producao
date          
1977   19256.0
1978   14017.0
1979   16514.0
1980   19435.0
1981   21284.0
1982   21605.0
1983   19015.0
1984   21178.0
1985   21175.0
1986   20265.0


fh: Optional[int or list or np.array], default = 1

The forecast horizon to be used for forecasting. Default is set to 1 i.e. forecast one point ahead. When integer is passed it means N continuous points in the future without any gap. If you want to forecast values with gaps, you must pass an array e.g. np.arange([13, 25]) will skip the first 12 future points and forecast from the 13th point till the 24th point ahead (note in numpy right value is inclusive and left is exclusive).

In [72]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=42, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,42
1,Target,producao
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(48, 1)"
5,Transformed data shape,"(48, 1)"
6,Transformed train set shape,"(47, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [73]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

## Checking Train-Test Split

In [74]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

## Comparing Models

The most basic assumption of time series forecasting is that past patterns continue into the future. 
Based on this assumption, in the short-term future forecast, since the environment in which the data is generated is similar to the present, 
the uncertainty is small and the forecast can be well-predicted. However, as we move towards long-term future forecasting, the possibility of the environment in which data is generated is likely to change, and unpredictable situations occur, and as errors continue to accumulate, uncertainty about the forecast data is bound to increase.

We compare the performance of various models and try to select the 4 models with the best performance.

In [75]:
top4 = exp.compare_models(n_select = 4)

## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data. 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.

## Difference Plot Using Order

In [11]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":50})

### theta 

In [76]:
theta = exp.create_model('theta')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = theta, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 6},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(theta, fh = 6)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,2.3737,1.6817,14644.0852,14644.0852,0.1681,0.1551
1,2021.0000,2.9675,2.1230,18936.0601,18936.0601,0.1674,0.1827
2,2022.0000,3.6853,2.6075,25125.2204,25125.2204,0.1905,0.2106
Mean,nan,3.0088,2.1374,19568.4552,19568.4552,0.1753,0.1828
SD,nan,0.5362,0.3781,4302.2081,4302.2081,0.0107,0.0226


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2024,125519.2050
2025,126505.2741
2026,127491.3431
2027,128477.4122
2028,129463.4813
2029,130449.5504


###  ADA

In [77]:
auto_arima = exp.create_model('auto_arima')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = auto_arima, plot = 'forecast', 
               data_kwargs = {'fh' : 7},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(auto_arima, fh = 7)


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,2.7998,1.9836,17272.7915,17272.7915,0.1983,0.1804
1,2021.0000,2.2550,1.6132,14389.0673,14389.0673,0.1272,0.1358
2,2022.0000,4.7019,3.3268,32056.2981,32056.2981,0.2430,0.2767
Mean,nan,3.2522,2.3079,21239.3856,21239.3856,0.1895,0.1976
SD,nan,1.0489,0.7362,7738.7838,7738.7838,0.0477,0.0588


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2024,121024.7563
2025,126592.7160
2026,132230.2619
2027,137937.3941
2028,143714.1126
2029,149560.4173
2030,155476.3082


## exp_smooth

In [14]:
exp_smooth  = exp.create_model('exp_smooth')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 6},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,2.8110,1.9915,17341.4091,17341.4091,0.1991,0.1811
1,2021.0000,1.5896,1.1372,10143.2527,10143.2527,0.0897,0.0939
2,2022.0000,3.2489,2.2987,22150.0505,22150.0505,0.1679,0.1833
Mean,nan,2.5498,1.8091,16544.9041,16544.9041,0.1522,0.1528
SD,nan,0.7021,0.4914,4934.0054,4934.0054,0.0460,0.0417


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## ets

In [15]:

ets = exp.create_model('ets')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = ets, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 6},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,2.9277,2.0742,18061.5242,18061.5242,0.2074,0.1879
1,2021.0000,1.5866,1.1351,10124.3440,10124.3440,0.0895,0.0937
2,2022.0000,3.2062,2.2686,21859.4735,21859.4735,0.1657,0.1807
Mean,nan,2.5735,1.8259,16681.7806,16681.7806,0.1542,0.1541
SD,nan,0.7070,0.4949,4889.1775,4889.1775,0.0488,0.0428


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## ALL GRAPHS

In [16]:
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = ada_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})              

NameError: name 'ada_cds_dt' is not defined

## Avaliando a Produtividade

In [78]:
#função para tratar campo data                                                                                                                                                                      
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('dbmilho_br.xls', sheet_name='Planilha1')
print(df.head(10))

   date  producao  produtividade     area
0  1977   19256.0           1632  11797.3
1  1978   14017.0           1276  10985.1
2  1979   16514.0           1461  11304.8
3  1980   19435.0           1665  11669.9
4  1981   21284.0           1752  12147.1
5  1982   21605.0           1692  12771.1
6  1983   19015.0           1631  11658.2
7  1984   21178.0           1735  12205.6
8  1985   21175.0           1773  11940.2
9  1986   20265.0           1549  13083.5


In [79]:
df.drop(['producao','area'], axis = 1,inplace=True)
df.head(10)

,date,produtividade
0,1977,1632
1,1978,1276
2,1979,1461
3,1980,1665
4,1981,1752
5,1982,1692
6,1983,1631
7,1984,1735
8,1985,1773
9,1986,1549


In [80]:
df.set_index('date',inplace=True)

In [81]:
df=df.sort_index(axis=0)
print(df.head(10))

      produtividade
date               
1977           1632
1978           1276
1979           1461
1980           1665
1981           1752
1982           1692
1983           1631
1984           1735
1985           1773
1986           1549


In [82]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=15, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,15
1,Target,produtividade
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(48, 1)"
5,Transformed data shape,"(48, 1)"
6,Transformed train set shape,"(47, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [83]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})


In [84]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})


In [85]:
top4 = exp.compare_models(n_select = 4)


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,TT (Sec)
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,1.3936,1.0270,476.2745,476.2745,0.1005,0.0940,0.0567
polytrend,Polynomial Trend Forecaster,1.4195,1.0467,489.0312,489.0312,0.1004,0.0958,0.0133
ada_cds_dt,AdaBoost w/ Cond. Deseasonalize & Detrending,1.6875,1.2434,572.2115,572.2115,0.1238,0.1114,0.0600
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,1.7327,1.2755,584.5487,584.5487,0.1286,0.1157,0.0633
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,1.7363,1.2795,598.5746,598.5746,0.1226,0.1168,0.0333
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,1.7409,1.2825,588.7891,588.7891,0.1285,0.1144,0.1000
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,1.8058,1.3305,622.7085,622.7085,0.1275,0.1215,0.0433
omp_cds_dt,Orthogonal Matching Pursuit w/ Cond. Deseasonalize & Detrending,1.8058,1.3305,622.7110,622.7110,0.1275,0.1215,0.0567
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,1.8058,1.3305,622.7093,622.7093,0.1275,0.1215,0.0467
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,1.8058,1.3305,622.7110,622.7110,0.1275,0.1215,0.0433


Processing:   0%|          | 0/112 [00:00<?, ?it/s]

## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data.
 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.

In [ ]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":50})


####  lightgbm_cds_dt

In [29]:
lightgbm_cds_dt = exp.create_model('lightgbm_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = lightgbm_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 6},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(lightgbm_cds_dt, fh = 6)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,2.8562,2.1048,951.3643,951.3643,0.2179,0.1965
1,2021.0000,0.3756,0.2753,132.2646,132.2646,0.0252,0.0256
2,2022.0000,0.9490,0.7008,345.1946,345.1946,0.0583,0.0600
Mean,nan,1.3936,1.0270,476.2745,476.2745,0.1005,0.0940
SD,nan,1.0603,0.7817,347.0038,347.0038,0.0841,0.0738


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2024,5579.9026
2025,5677.8264
2026,5775.7502
2027,5873.6740
2028,5971.5978
2029,6069.5216


### xgboost

In [86]:
xgboost_cds_dt = exp.create_model('xgboost_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = xgboost_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 7},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(xgboost_cds_dt, fh = 7)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,4.0809,3.0074,1359.3283,1359.3283,0.3113,0.2694
1,2021.0000,1.1871,0.8701,417.9844,417.9844,0.0797,0.0830
2,2022.0000,0.4932,0.3641,179.3786,179.3786,0.0303,0.0298
Mean,nan,1.9204,1.4139,652.2304,652.2304,0.1404,0.1274
SD,nan,1.5538,1.1455,509.3942,509.3942,0.1225,0.1027


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2024,6276.6407
2025,5059.5234
2026,6442.8578
2027,5738.2922
2028,6599.7779
2029,6418.8860
2030,5248.8039


#### polytrend

In [31]:
polytrend = exp.create_model('polytrend')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = polytrend, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,2.6429,1.9476,880.3266,880.3266,0.2016,0.1831
1,2021.0000,0.0742,0.0544,26.1323,26.1323,0.0050,0.0050
2,2022.0000,1.5413,1.1381,560.6348,560.6348,0.0947,0.0994
Mean,nan,1.4195,1.0467,489.0312,489.0312,0.1004,0.0958
SD,nan,1.0522,0.7756,352.3798,352.3798,0.0804,0.0728


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


### auto_Arima

In [35]:
auto_arima = exp.create_model('auto_arima')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = auto_arima, plot = 'forecast', 
               data_kwargs = {'fh' : 7},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(auto_arima, fh = 7)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,4.3731,3.2227,1456.6507,1456.6507,0.3336,0.2859
1,2021.0000,0.0687,0.0503,24.1737,24.1737,0.0046,0.0046
2,2022.0000,1.5922,1.1757,579.1328,579.1328,0.0978,0.1028
Mean,nan,2.0113,1.4829,686.6524,686.6524,0.1453,0.1311
SD,nan,1.7821,1.3132,589.7276,589.7276,0.1384,0.1166


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2024,5632.3597
2025,5724.2577
2026,5816.1556
2027,5908.0536
2028,5999.9516
2029,6091.8496
2030,6183.7476


### ALL GRAPHS

In [ ]:
exp.plot_model(estimator = lightgbm_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = xgboost_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = polytrend, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

In [48]:
## Avaliando a Area

In [52]:
#função para tratar campo data                                                                                                                                                                      
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('dbmilho_br.xls', sheet_name='Planilha1')
print(df.head(10))

   date  producao  produtividade     area
0  1977   19256.0           1632  11797.3
1  1978   14017.0           1276  10985.1
2  1979   16514.0           1461  11304.8
3  1980   19435.0           1665  11669.9
4  1981   21284.0           1752  12147.1
5  1982   21605.0           1692  12771.1
6  1983   19015.0           1631  11658.2
7  1984   21178.0           1735  12205.6
8  1985   21175.0           1773  11940.2
9  1986   20265.0           1549  13083.5


In [53]:
df.drop(['producao','produtividade'], axis = 1,inplace=True)
df.set_index('date',inplace=True)
df=df.sort_index(axis=0)
print(df.tail(10))

         area
date         
2015  15692.9
2016  15922.5
2017  17591.7
2018  16616.4
2019  17492.9
2020  18527.3
2021  19943.6
2022  21580.6
2023  22269.2
2024      NaN


In [56]:
df=df.dropna()
print(df.tail(10))

         area
date         
2014  15828.9
2015  15692.9
2016  15922.5
2017  17591.7
2018  16616.4
2019  17492.9
2020  18527.3
2021  19943.6
2022  21580.6
2023  22269.2


In [57]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=15, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,15
1,Target,area
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(47, 1)"
5,Transformed data shape,"(47, 1)"
6,Transformed train set shape,"(46, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [58]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

In [59]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

In [61]:
top4 = exp.compare_models(n_select = 4)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,TT (Sec)
exp_smooth,Exponential Smoothing,0.8005,0.6865,929.1953,929.1953,0.0482,0.0499,0.0567
theta,Theta Forecaster,0.8628,0.7385,1007.9410,1007.9410,0.0517,0.0534,0.0300
ets,ETS,0.9606,0.8222,1123.6967,1123.6967,0.0575,0.0595,0.0400
auto_arima,Auto ARIMA,1.0488,0.8941,1241.4977,1241.4977,0.0623,0.0644,0.9467
naive,Naive Forecaster,1.1420,0.9707,1362.5667,1362.5667,0.0676,0.0700,0.0300
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,1.6599,1.4118,1977.3205,1977.3205,0.0983,0.1034,0.0533
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,1.6599,1.4118,1977.3199,1977.3199,0.0983,0.1034,0.0533
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,1.6599,1.4118,1977.3201,1977.3201,0.0983,0.1034,0.0533
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,1.6599,1.4118,1977.3200,1977.3200,0.0983,0.1034,0.0500
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,1.6599,1.4118,1977.3199,1977.3199,0.0983,0.1034,0.0967


Processing:   0%|          | 0/120 [00:00<?, ?it/s]


## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data.
 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.


In [43]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":50})

In [62]:
exp_smooth = exp.create_model('exp_smooth')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(exp_smooth, fh = 8)

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2023,20549.4258
2024,20724.0199
2025,20449.3996
2026,21326.3979
2027,22586.1470
2028,21546.2980
2029,21719.7015
2030,21422.5363


In [63]:
theta = exp.create_model('theta')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(theta, fh = 8)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019.0000,1.3906,1.2008,1576.8117,1576.8117,0.0851,0.0889
1,2020.0000,0.6838,0.5863,805.2358,805.2358,0.0404,0.0412
2,2021.0000,0.5140,0.4285,641.7756,641.7756,0.0297,0.0302
Mean,nan,0.8628,0.7385,1007.9410,1007.9410,0.0517,0.0534
SD,nan,0.3796,0.3332,407.7501,407.7501,0.0240,0.0255


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2023,20503.5134
2024,20777.3217
2025,20305.6457
2026,21006.7883
2027,21994.5313
2028,20831.9460
2029,21109.0774
2030,20628.8380


In [64]:
ets= exp.create_model('ets')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(ets, fh = 8)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019.0000,1.3878,1.1984,1573.6449,1573.6449,0.0849,0.0887
1,2020.0000,0.9572,0.8207,1127.1231,1127.1231,0.0565,0.0582
2,2021.0000,0.5369,0.4476,670.3221,670.3221,0.0311,0.0316
Mean,nan,0.9606,0.8222,1123.6967,1123.6967,0.0575,0.0595
SD,nan,0.3474,0.3065,368.7880,368.7880,0.0220,0.0234


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2023,20413.4541
2024,20760.0601
2025,20461.8766
2026,21302.5478
2027,22536.7229
2028,21341.1744
2029,21695.0342
2030,21375.1947


In [65]:
auto_arima= exp.create_model('auto_arima')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = auto_arima, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(auto_arima, fh = 8)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019.0000,1.0694,0.9234,1212.5506,1212.5506,0.0654,0.0677
1,2020.0000,1.1473,0.9837,1350.9898,1350.9898,0.0677,0.0701
2,2021.0000,0.9298,0.7751,1160.9527,1160.9527,0.0538,0.0553
Mean,nan,1.0488,0.8941,1241.4977,1241.4977,0.0623,0.0644
SD,nan,0.0900,0.0876,80.2370,80.2370,0.0061,0.0065


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2023,21286.0355
2024,21550.7600
2025,21863.1742
2026,22290.9316
2027,22785.3458
2028,22696.3800
2029,22776.3333
2030,22870.6901
